In [7]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")


COLOR_LIST1 = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

In [8]:
adht_ward_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "counts_ward.csv" ), parse_dates=['date'])

date_start = pd.to_datetime('2020-02-01')
date_end   = pd.to_datetime('2021-02-28')
dates_sim  = pd.date_range(date_start, date_end)

adht_ward_df   = adht_ward_df[adht_ward_df.date.isin(dates_sim)]
#selected_ward = ['Allen Hospital', 'Harkness Pavilion', 'Milstein Hospital', 'Mschony', 'Presbyterian Hospital']

A_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_admitted')
D_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_discharged')
H_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_hospitalized')
tests_df = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_tested')

pop        = H_df.mean(axis=1)
num_pop    = len(pop)
ward_names = pop.index

ward_num          = len(ward_names)
ward_transfers_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "transfers_ward.csv"), parse_dates=['date'])
ward_transfers_df = ward_transfers_df[ward_transfers_df.date.isin(dates_sim)]

M_df = np.zeros((ward_num, ward_num, len(dates_sim)+1))

for i in range(ward_num):
    ward_from = ward_names[i]
    for j in range(ward_num):
        ward_to      = ward_names[j]
        transfers_ij = ward_transfers_df[(ward_transfers_df.ward_from==ward_from) & (ward_transfers_df.ward_to==ward_to)]

        if(transfers_ij.shape[0] > 0) :
            dates_ij                = transfers_ij.date.values
            dates_ind               = np.where(np.in1d(dates_ij, dates_sim))[0]
            transfered              = transfers_ij.num_transfered.values
            M_df[i, j, dates_ind-1] = transfered

In [ ]:
ward_transfers_df

,date,ward_from,ward_to,num_transfered
396,2020-03-01,Allen-Allen Hospital-Aln Emergency,Columbia-Presbyterian Hospital-Pby Adult Emerg...,1
397,2020-03-01,Chony-Mschony-Msch Emergency,Columbia-Presbyterian Hospital-Pby Adult Emerg...,1
398,2020-03-02,Allen-Allen Hospital-Aln Emergency,Columbia-Milstein Hospital-Pby Adult Emergency,1
399,2020-03-02,Allen-Allen Hospital-Aln Emergency,Columbia-Presbyterian Hospital-Pby Adult Emerg...,1
400,2020-03-02,Columbia-Milstein Hospital-Mil 6 Hudson North,Columbia-Herbert Irving Pavilion-Hip 14 Adult ...,1
...,...,...,...,...
6775,2021-02-27,Columbia-Presbyterian Hospital-Pby 14 Transpla...,Columbia-Presbyterian Hospital-Pby Adult Emerg...,1
6776,2021-02-27,Columbia-Vanderbilt Clinic-Vc Col Path Lab Out...,Columbia-Milstein Hospital-Mil 5 Ccu,1
6777,2021-02-27,Columbia-Vanderbilt Clinic-Vc Col Path Lab Out...,Columbia-Milstein Hospital-Mil 6 Hudson South,1
6778,2021-02-28,Allen-Allen Hospital-Aln Emergency,Columbia-Milstein Hospital-Pby Adult Emergency,1
